In [ ]:
from __future__ import print_function 
koundinya_challa = "979e3598-5fc4-11ea-af52-0201714f6eab" 
UNCG = "90c62273-3c24-4bc6-b963-3f6c9b4f294e"      
tutorial_users_group = "e4c16d15-6d04-11e5-ba46-22000b92c6ec"     

In [ ]:
import globus_sdk

In [ ]:
CLIENT_ID = 'c59dcb05-62bf-40e3-900a-60c3935644a6'

In [ ]:
native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
native_auth_client.oauth2_start_flow()
print("Login Here:\n\n{0}".format(native_auth_client.oauth2_get_authorize_url()))
print("\nIMPORTANT NOTE: the link above can only be used once!")
print("If login or a later step in the flow fails, you must execute this cell again to generate a new link.")

In [ ]:
auth_code = "W7ImAQJQR8mVTEeH4GbrRy9MZJkz5S"
token_response = native_auth_client.oauth2_exchange_code_for_tokens(auth_code)
transfer_access_token = token_response.by_resource_server['transfer.api.globus.org']['access_token']
transfer_authorizer = globus_sdk.AccessTokenAuthorizer(transfer_access_token)
tc = globus_sdk.TransferClient(authorizer=transfer_authorizer)

In [ ]:
r = tc.endpoint_search(filter_scope="my-endpoints")
for ep in r:
    print("{} ({})".format(ep["display_name"] or ep["canonical_name"], ep["id"]))

In [ ]:
r = tc.get_endpoint(koundinya_challa)
print("HTTP Status Code:", r.http_status)
print("Content Type:", r.content_type)
print("Data:", r)
print("Endpoint Display Name:", r["display_name"]) 

In [ ]:
endpoint_id = koundinya_challa
endpoint_path = "/~/S/UNCG/"
r = tc.operation_ls(endpoint_id, path=endpoint_path)
print("==== Endpoint_ls for endpoint {} {} ====".format(endpoint_id, endpoint_path))
for item in r:
    print("{}: {} [{}]".format(item["type"], item["name"], item["size"]))

In [ ]:
try:
    endpoint_id = koundinya_challa
    endpoint_path = "/~/S/UNCG/Globus_playground/example-dir"
    mkdir_result = tc.operation_mkdir(endpoint_id, path=endpoint_path)
    print(mkdir_result["message"])
except globus_sdk.GlobusAPIError as ex:
    print("Error code:", ex.code)
    print("Error message:", ex.message)

In [ ]:
try:
    endpoint_id = koundinya_challa
    r = tc.operation_rename(endpoint_id, oldpath="/~/S/UNCG/Globus_playground/example-dir",
                            newpath="/~/S/UNCG/Globus_playground/example-dir-renamed")
    print(r["message"])
except globus_sdk.GlobusAPIError as ex:
    print("Error code:", ex.code)
    print("Error message:", ex.message)

In [ ]:
source_endpoint_id = koundinya_challa
source_path = "/~/S/UNCG/Globus_playground/gcrNet"

dest_endpoint_id = UNCG
dest_path = "/~/"

label = "My example transfer"

tdata = globus_sdk.TransferData(tc, source_endpoint_id,
                                dest_endpoint_id,
                                label=label)

tdata.add_item(source_path, dest_path, recursive=True)

tc.endpoint_autoactivate(source_endpoint_id)
tc.endpoint_autoactivate(dest_endpoint_id)

submit_result = tc.submit_transfer(tdata)
print("Task ID:", submit_result["task_id"])